# Analyze MTP Log Results

## Input Required Information

| Variable | Use |
| --- | --- |
| LOG_DIR_PATH | Location of the log directory. | 

In [ ]:
LOG_DIR_PATH = "/home/fabric/work/custom/FABRIC-Automation/logs/mtp_logs"
END_TIME = 1714158888103
import os # Used to iterate over log directory

## **Control Overhead**

In [ ]:
def control_overhead_helper(file_name):
    f = open(file_name,"r")
   
    line = f.readline()
    size = 0

    while line:
        if "FAILURE UPDATE message received" in line:
            time = int(line.split(" ")[6].replace(",",""))
            line = f.readline()
            size += int(line.split("=")[1]) if time < END_TIME else 0

        line = f.readline()   
    
    f.close()

    return size

# Default starting value for the number of bytes
total_size = 0

files = os.listdir(LOG_DIR_PATH)
for file in files:
    if("MTP" in file):
        filePath = os.path.join(LOG_DIR_PATH, file)

        if os.path.getsize(filePath) > 0:
            total_size += control_overhead_helper(filePath)

print(f"Control overhead (byte) = {total_size}")

## **Convergence Timing**

In [ ]:
def convergent_time_helper(file_name):
    f = open(file_name,"r")

    line = f.readline()

    while line:
        token = line.split(" ")
        
        if "Detected a failure" in line:            
            return 0,int(token[len(token)-1])
        
        elif "FAILURE UPDATE message received" in line:
            time = int(token[6].replace(",",""))

            if(time < END_TIME):
                return 1,int(time)
        
        line = f.readline()
        
    return -1,-1

# Default starting values for the timing range
begin_time = -1
end_time = -1

files = os.listdir(LOG_DIR_PATH)
for file in files:
    if("MTP" in file):
        filePath = os.path.join(LOG_DIR_PATH, file)

        if os.path.getsize(filePath) > 0:
            type, value = convergent_time_helper(filePath)

            if type == 0:
                if(begin_time > 0):
                    raise Exception("Multiple begin times, please check log")
                begin_time = value

            else:
                end_time = max(end_time,value)

convergenceTime = end_time - begin_time
print(f"End time = {end_time}\nBegin time = {begin_time}")
print(f"Convergent time (ms) = {convergenceTime}")

## **Churn Rate (Number of Changes)**

In [ ]:
def number_of_change_helper(file_name):
    f = open(file_name,"r")

    line = f.readline()

    while line:
        if ("Detected a failure" in line):
            time = int(line.split(" ")[9])
            if(time < END_TIME):
                return 1

        elif("FAILURE UPDATE message received" in line):
            time = int(line.split(" ")[6].replace(",",""))
            if(time < END_TIME):
                return 1
        
        line = f.readline()
    
    return 0

# Default starting value for the number of changes that occur
change_count = 0

files = os.listdir(LOG_DIR_PATH)
for file in files:
    if("MTP" in file):
        filePath = os.path.join(LOG_DIR_PATH, file)

        if os.path.getsize(filePath) > 0:
            change_count += number_of_change_helper(filePath)

print(f"Number of change = {change_count}")

## Manual Log Entry Checks

This section is not something you need to use for each experiment run. It's only for log debugging purposes if you notice weird results or want to only view how certain nodes reacted to whatever you changed in the topology.

| Variable | Use |
| --- | --- |
| PREFIX_TO_CHECK | A list containing however many name prefixes you'd like to search through for logs. If you only want one prefix, just include one list element with the name. | 
| LINE_MATCH | A string containing log information (ex: "Detected a failure") that will be search for and printed out | 

In [ ]:
PREFIX_TO_CHECK = ["L", "T", "S"]
LINE_MATCH = "Detected a failure"

In [ ]:
files = os.listdir(LOG_DIR_PATH)

# Search through the entire log directory
for file in files:
    
    # If there is a prefix match for a log file name (which includes the node name), check it.
    if any(prefix in file for prefix in PREFIX_TO_CHECK):
        filePath = os.path.join(LOG_DIR_PATH, file)
        print(f"\n========== {filePath} ==========")
        
        f = open(filePath,"r") # Open log file
        
        # Read each line and find log line matches, if there are any. 
        line = f.readline()
        while line:
            if LINE_MATCH in line:
                print(line)
            line = f.readline()

        f.close() # Open log file
        